In [177]:
# Dependencies.
import pandas as pd
from config import client_id, client_secret
from params import key_signatures, modes
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from pprint import pprint
import numpy as np

# Load variables to environment.
os.environ['SPOTIPY_CLIENT_ID'] = client_id
os.environ['SPOTIPY_CLIENT_SECRET'] = client_secret

In [178]:
# Function to obtain user's input.
def get_input():

    user_input = input('Enter the playlist URI: ')

    return user_input

In [179]:
# Function to perform API call and return results in JSON format.
def get_results(t, o):

    sp = spotipy.Spotify(client_credentials_manager = SpotifyClientCredentials())

    # Check if string contains 'https', the ref['next'] will contain this if the list has more than 100 songs.
    if 'https' in t:

        # Split the string in URL using '/'
        split = t.split('/')

        # Split list index 5 will contain part of URI, multiple o by 100 for the offset in the function call.
        results = sp.playlist_tracks('spotify:playlist:' + split[5], offset = o * 100)

    else:
        results = sp.playlist_tracks(t)

    return results

In [180]:
# Function to create dict with data.
def create_df(results):

    # Empty dict to hold data from Spotify.
    data = {
        'title': [],
        'artist': [],
        'album': [], 
        'spotify_uri': []
    }

    # Loop control.
    loop = True

    # Results as json object.
    ref = results

    # Offset variable.
    offset = 0

    while loop:

        # Adds values to dict.
        for song in ref['items']:
            data['title'].append(song['track']['name'])
            data['artist'].append(song['track']['artists'][0]['name'])
            data['album'].append(song['track']['album']['name'])
            data['spotify_uri'].append(song['track']['uri'])

        # Check for ref['next'], if there's text then playlist has more items.
        if ref['next'] is not None:
            offset += 1
            ref = get_results(ref['next'], offset)
        
        # Terminate loop.
        else:
            loop = False

    return pd.DataFrame(data)

In [181]:
# Function to perform API call for audio features and return results in JSON format.
def get_features(t):

    sp = spotipy.Spotify(client_credentials_manager = SpotifyClientCredentials())

    results = sp.audio_features(t)

    return results

In [182]:
# Function to find the time key signatures of the song.
def key_time_finder(results):

    try:
        for key, value in modes.items():
            if results[0]['mode'] == key:
                m = value

        for key, value in key_signatures.items():
            if results[0]['key'] == key:
                k = value[results[0]['mode']]

        t = str(results[0]['time_signature'])
    
    except IndexError:
        
        return np.NaN, np.NaN
    
    return k + " " + m, t + '/4'

In [183]:
results = get_results(get_input(), 1)
df = create_df(results)

uris = list(df['spotify_uri'])

key_col = []
time_col = []

for uri in uris:
    response = get_features(uri)
    ks, ts = key_time_finder(response)
    key_col.append(ks)
    time_col.append(ts)

df['key_signature'] = key_col
df['time_signature'] = time_col


df = df.dropna()
df

,title,artist,album,spotify_uri,key_signature,time_signature
0,I Saw Her Standing There - Remastered 2009,The Beatles,Please Please Me (Remastered),spotify:track:3KiexfmhxHvG5IgAElmTkd,E major,4/4
1,Misery - Remastered 2009,The Beatles,Please Please Me (Remastered),spotify:track:40qXGg5nRbcWzcFb26KWkQ,C major,4/4
2,Anna (Go To Him) - Remastered 2009,The Beatles,Please Please Me (Remastered),spotify:track:2baEFuU0gQon0hgVRioI1o,D major,4/4
3,Chains - Remastered 2009,The Beatles,Please Please Me (Remastered),spotify:track:3JQWLa88R35d971o5bIImd,Bb major,4/4
4,Boys - Remastered 2009,The Beatles,Please Please Me (Remastered),spotify:track:7JxGM1R32ZqfwZou3VtnTg,E major,4/4
...,...,...,...,...,...,...
435,Strawberry Fields Forever - Take 7,The Beatles,Sgt. Pepper's Lonely Hearts Club Band (Deluxe ...,spotify:track:766Skoghn3BUAVGrUpJ7n9,A minor,4/4
436,Strawberry Fields Forever - Take 26,The Beatles,Sgt. Pepper's Lonely Hearts Club Band (Deluxe ...,spotify:track:1Mkt1BqBK9D02g1iMmRkqq,C major,4/4
437,Strawberry Fields Forever - Stereo Mix 2015,The Beatles,Sgt. Pepper's Lonely Hearts Club Band (Deluxe ...,spotify:track:6B89krM9fveE6RqAWmonK9,Bb major,4/4
438,Penny Lane - Take 6 / Instrumental,The Beatles,Sgt. Pepper's Lonely Hearts Club Band (Deluxe ...,spotify:track:1Spno7RVP9d3wszxojPJXZ,C major,3/4


In [184]:
df['key_signature'].value_counts()

D major        69
C major        60
A major        59
E major        34
F major        26
A minor        16
E minor        13
B minor        13
Ab major       12
F# minor       11
C# minor       10
C#/Db major     9
Bb major        8
D minor         7
B major         5
F#/Gb major     5
G#/Ab minor     5
C minor         4
Eb major        3
F minor         3
D#/Eb minor     2
G, G minor      1
Name: key_signature, dtype: int64

In [185]:
df.to_csv('output.csv', index = False)